In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
# from tqdm import tqdm

In [8]:
!pip install tqdm

In [25]:
url = 'https://www.sta1.com/shops?gndr=F&shopType=S&pg=10'
r = requests.get(url)

In [26]:
soup = bs(r.text, 'lxml')

In [27]:
shop_list = []
for i in soup.select('.shop-list > li')[2:]:
    shop_info = {}
    shop_info['rank'] = i.select_one('.rank').text
    shop_info['title'] = i.select_one('.shop-name').text
    shop_info['age'] = i.select_one('.info p:nth-child(3)').text
    shop_info['genre'] = i.select_one('.info p:nth-child(4)').text
    
    shop_list.append(shop_info)

In [28]:
shop_list

[{'rank': '2위', 'title': '소울메이트', 'age': '10대,20대,30대', 'genre': '언더웨어'},
 {'rank': '3위', 'title': '젝시믹스', 'age': '10대,20대,30대', 'genre': '피트니스'},
 {'rank': '4위', 'title': '리린', 'age': '30대', 'genre': '미시스타일,페미닌'},
 {'rank': '5위', 'title': '안나앤모드', 'age': '30대', 'genre': '미시스타일,심플베이직'},
 {'rank': '6위', 'title': '안나키즈', 'age': '30대', 'genre': '미시스타일,심플베이직'},
 {'rank': '7위', 'title': '스토리나인', 'age': '30대', 'genre': '미시스타일,심플베이직'},
 {'rank': '8위', 'title': '핫핑', 'age': '10대,20대', 'genre': '빅사이즈,심플베이직'},
 {'rank': '9위', 'title': '공구우먼', 'age': '20대,30대', 'genre': '빅사이즈,심플베이직'},
 {'rank': '10위', 'title': '썸제이', 'age': '30대', 'genre': '미시스타일,심플베이직'},
 {'rank': '11위', 'title': '베니토', 'age': '20대,30대', 'genre': '럭셔리·명품,페미닌'},
 {'rank': '12위', 'title': '메이블루', 'age': '30대', 'genre': '미시스타일,심플베이직'},
 {'rank': '13위', 'title': '리틀블랙', 'age': '30대', 'genre': '모던시크,미시스타일'},
 {'rank': '14위', 'title': '안나앤블루', 'age': '30대', 'genre': '미시스타일,심플베이직'},
 {'rank': '15위', 'title': '로즈베이', 'age': '30대', 'genr

In [32]:
df = pd.DataFrame(shop_list)

## 코사인유사도 구하기

In [30]:
# 문자열 카운터 벡터라이즈
from sklearn.feature_extraction.text import CountVectorizer

In [33]:
cv = CountVectorizer(ngram_range=(1,2))
df_mat = cv.fit_transform(df['genre'])
print(df_mat)

  (0, 47)	1
  (1, 61)	1
  (2, 26)	1
  (2, 59)	1
  (2, 29)	1
  (3, 26)	1
  (3, 42)	1
  (3, 28)	1
  (4, 26)	1
  (4, 42)	1
  (4, 28)	1
  (5, 26)	1
  (5, 42)	1
  (5, 28)	1
  (6, 42)	1
  (6, 32)	1
  (6, 33)	1
  (7, 42)	1
  (7, 32)	1
  (7, 33)	1
  (8, 26)	1
  (8, 42)	1
  (8, 28)	1
  (9, 59)	1
  (9, 13)	1
  :	:
  (590, 26)	1
  (590, 42)	1
  (590, 28)	1
  (591, 0)	1
  (592, 42)	1
  (592, 21)	1
  (592, 23)	1
  (593, 42)	1
  (593, 49)	1
  (593, 51)	1
  (594, 55)	1
  (594, 30)	1
  (594, 56)	1
  (595, 21)	1
  (595, 2)	1
  (595, 3)	1
  (596, 42)	1
  (596, 49)	1
  (596, 51)	1
  (597, 21)	1
  (597, 49)	1
  (597, 24)	1
  (598, 42)	1
  (598, 49)	1
  (598, 51)	1


In [34]:
from sklearn.metrics.pairwise import cosine_similarity

In [35]:
df_sim = cosine_similarity(df_mat, df_mat)
print(df_sim.shape)
print(df_sim[:2])

(599, 599)
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [36]:
sorted_df_sim = df_sim.argsort()[:, ::-1]

In [37]:
sorted_df_sim

array([[  0, 585, 437, ..., 390, 389, 299],
       [504, 101,   1, ..., 394, 393,   0],
       [ 93, 118,  42, ..., 389, 390,   0],
       ...,
       [598,  40, 345, ..., 471, 263,   0],
       [145, 327, 575, ..., 372, 373,   0],
       [598,  40, 345, ..., 471, 263,   0]], dtype=int64)

In [38]:
def find_df_sim(df, sorted_df_sim, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    similar_indexes = sorted_df_sim[title_index, : (top_n)]
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    return df.iloc[similar_indexes]

In [40]:
find_df_sim(df, sorted_df_sim, "스토리나인", 10)

[[299  52 552  92 541  95  97 116 121 122]]


,rank,title,age,genre
299,301위,초코맘,30대,"미시스타일,심플베이직"
52,54위,아이팜므,30대,"미시스타일,심플베이직"
552,554위,얀스토리,30대,"미시스타일,심플베이직"
92,94위,써니11,30대,"미시스타일,심플베이직"
541,543위,라망,30대,"미시스타일,심플베이직"
95,97위,시크폭스,30대,"미시스타일,심플베이직"
97,99위,안녕윤수야,30대,"미시스타일,심플베이직"
116,118위,코코제인,30대,"미시스타일,심플베이직"
121,123위,한스스타일,30대,"미시스타일,심플베이직"
122,124위,루노,30대,"미시스타일,심플베이직"
